In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

In [3]:
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save 
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
train_subset = 10000
graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset,:])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)

In [6]:
num_steps = 801

def accuracy(predictions,labels):
    return ((100.0*np.sum(np.argmax(predictions,1)==np.argmax(labels,1)))/predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Intialized')
    for step in range(num_steps):
        _,l,predictions = session.run([optimizer,loss,train_prediction])
        if step%100==0:
            print('Loss at',str(step)+'th step: '+str(l))
            print('Training Accuracy: '+str(accuracy(predictions,train_labels[:train_subset,:]))+'%')
            print('Validation Accuracy: '+str(accuracy(valid_prediction.eval(),valid_labels))+'%')
            print('\n')
    print('Test Accuracy: '+str(accuracy(test_prediction.eval(),test_labels))+'%')

Intialized
Loss at 0th step: 18.0243
Training Accuracy: 9.91%
Validation Accuracy: 12.1%


Loss at 100th step: 2.31926
Training Accuracy: 71.8%
Validation Accuracy: 70.46%


Loss at 200th step: 1.86376
Training Accuracy: 74.63%
Validation Accuracy: 72.98%


Loss at 300th step: 1.61119
Training Accuracy: 75.78%
Validation Accuracy: 73.96%


Loss at 400th step: 1.44145
Training Accuracy: 76.69%
Validation Accuracy: 74.64%


Loss at 500th step: 1.31735
Training Accuracy: 77.46%
Validation Accuracy: 74.82%


Loss at 600th step: 1.2212
Training Accuracy: 78.16%
Validation Accuracy: 75.01%


Loss at 700th step: 1.1438
Training Accuracy: 78.69%
Validation Accuracy: 75.32%


Loss at 800th step: 1.07975
Training Accuracy: 79.13%
Validation Accuracy: 75.35%


Test Accuracy: 82.6105533006%


In [11]:
batch_size=128

graph=tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset,weights)+biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)

In [12]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized\n')
    for step in range(num_steps):
        offset = (batch_size*step)%(train_labels.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if step%500==0:
            print('Minibatch Loss at step '+str(step)+': '+str(l))
            print('Minibatch Accuracy: '+str(accuracy(predictions,batch_labels))+'%')
            print('Validation Accuracy: '+str(accuracy(valid_prediction.eval(),valid_labels))+'%')
            print('\n')
    print('Test Accuracy: '+str(accuracy(test_prediction.eval(),test_labels))+'%')

Initialized

Minibatch Loss at step 0: 15.2654
Minibatch Accuracy: 10.15625%
Validation Accuracy: 10.48%


Minibatch Loss at step 500: 1.74213
Minibatch Accuracy: 70.3125%
Validation Accuracy: 75.55%


Minibatch Loss at step 1000: 1.01792
Minibatch Accuracy: 78.125%
Validation Accuracy: 76.61%


Minibatch Loss at step 1500: 1.11873
Minibatch Accuracy: 78.125%
Validation Accuracy: 76.98%


Minibatch Loss at step 2000: 0.874594
Minibatch Accuracy: 74.21875%
Validation Accuracy: 77.43%


Minibatch Loss at step 2500: 0.694146
Minibatch Accuracy: 81.25%
Validation Accuracy: 78.37%


Minibatch Loss at step 3000: 0.951319
Minibatch Accuracy: 80.46875%
Validation Accuracy: 78.42%


Test Accuracy: 85.9218115787%


In [14]:
batch_size=128

graph=tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
    biases1 = tf.Variable(tf.zeros([1024]))
    
    hidden_relu = tf.nn.relu(tf.matmul(tf_train_dataset,weights1)+biases1)
    
    weights2 = tf.Variable(tf.truncated_normal([1024,num_labels]))
    biases2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    
    
    logits = tf.matmul(hidden_relu,weights2)+biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights1)+biases1),weights2)+biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights1)+biases1),weights2)+biases2)

In [15]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized\n')
    for step in range(num_steps):
        offset = (batch_size*step)%(train_labels.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if step%500==0:
            print('Minibatch Loss at step '+str(step)+': '+str(l))
            print('Minibatch Accuracy: '+str(accuracy(predictions,batch_labels))+'%')
            print('Validation Accuracy: '+str(accuracy(valid_prediction.eval(),valid_labels))+'%')
            print('\n')
    print('Test Accuracy: '+str(accuracy(test_prediction.eval(),test_labels))+'%')

Initialized

Minibatch Loss at step 0: 310.456
Minibatch Accuracy: 11.71875%
Validation Accuracy: 29.87%


Minibatch Loss at step 500: 17.988
Minibatch Accuracy: 79.6875%
Validation Accuracy: 80.25%


Minibatch Loss at step 1000: 11.3474
Minibatch Accuracy: 82.03125%
Validation Accuracy: 80.53%


Minibatch Loss at step 1500: 7.24724
Minibatch Accuracy: 77.34375%
Validation Accuracy: 73.67%


Minibatch Loss at step 2000: 6.15726
Minibatch Accuracy: 82.03125%
Validation Accuracy: 81.78%


Minibatch Loss at step 2500: 2.76865
Minibatch Accuracy: 89.0625%
Validation Accuracy: 81.87%


Minibatch Loss at step 3000: 6.69691
Minibatch Accuracy: 84.375%
Validation Accuracy: 81.65%


Test Accuracy: 88.6883144627%
